In [47]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyod.models.knn import KNN
from pyod.utils.utility import standardizer

from sklearn.neighbors import kneighbors_graph
from collections import Counter

In [2]:
root_path="../Projectors_publication/projector_Conso_CVAE_5couches-Day_WorkingDays_L1/"
sess= tf.Session()
saver = tf.train.import_meta_graph(root_path + "tf_data.ckpt.meta")
saver.restore(sess,root_path+"tf_data.ckpt")

INFO:tensorflow:Restoring parameters from ../Projectors_publication/projector_Conso_CVAE_5couches-Day_WorkingDays_L1/tf_data.ckpt


In [3]:
tf.get_variable('Variable_2', shape=(1500, 4))
embedding = sess.run('Variable_2:0')

In [4]:
embedding.shape

(1500, 4)

In [7]:
embedding_norm = standardizer(embedding)

In [40]:
contimation_range = range(1,11,1)

o_list = []

for c in contimation_range:
    c = c/100
    print("contamination level : {}".format(c))
    clf = KNN(n_neighbors=1, contamination=c)
    outliers =np.where(clf.fit_predict(embedding_norm) != 0)[0]
    print('With the {} we obtain {} outliers'.format('KNN classifier',len(outliers)))
    o_list += list(outliers)

contamination level : 0.01
With the KNN classifier we obtain 14 outliers
contamination level : 0.02
With the KNN classifier we obtain 30 outliers
contamination level : 0.03
With the KNN classifier we obtain 45 outliers
contamination level : 0.04
With the KNN classifier we obtain 60 outliers
contamination level : 0.05
With the KNN classifier we obtain 75 outliers
contamination level : 0.06
With the KNN classifier we obtain 90 outliers
contamination level : 0.07
With the KNN classifier we obtain 105 outliers
contamination level : 0.08
With the KNN classifier we obtain 120 outliers
contamination level : 0.09
With the KNN classifier we obtain 135 outliers
contamination level : 0.1
With the KNN classifier we obtain 150 outliers


In [46]:
outliers_freq_dict=Counter(o_list)
outliers_freq_dict

Counter({5: 10,
         57: 10,
         107: 10,
         518: 10,
         601: 10,
         730: 10,
         797: 10,
         929: 10,
         950: 10,
         956: 10,
         957: 10,
         1223: 10,
         1225: 10,
         1482: 10,
         65: 9,
         231: 9,
         240: 9,
         435: 9,
         437: 9,
         493: 9,
         497: 9,
         505: 9,
         566: 9,
         604: 9,
         675: 9,
         1199: 9,
         1245: 9,
         1336: 9,
         1483: 9,
         1487: 9,
         66: 8,
         119: 8,
         132: 8,
         596: 8,
         703: 8,
         771: 8,
         798: 8,
         799: 8,
         815: 8,
         830: 8,
         854: 8,
         1058: 8,
         1100: 8,
         1295: 8,
         1485: 8,
         22: 7,
         93: 7,
         455: 7,
         457: 7,
         478: 7,
         594: 7,
         705: 7,
         829: 7,
         831: 7,
         928: 7,
         960: 7,
         968: 7,
         979

In [68]:
outliers_df = pd.DataFrame(index=outliers_freq_dict.keys(), data=outliers_freq_dict.values())
outliers_df.columns = ["frequency"]
selected_indexes = outliers_df.index
outliers_df.head()

,frequency
5,10
57,10
107,10
518,10
601,10


In [85]:
# add date and manual label data
df_labels = pd.read_csv(root_path + 'df_labels.tsv', sep='\t').iloc[selected_indexes]
df_labels.head()

,Date,MaxTemperature,MinTemperature,Month,WeekDay,is_WeekDay,Holiday,Index,OddWeekday,OddHoliday,OddTemp,OddNeighbor,HD_predicted,nonWorkingDay,ToTag
5,2013-01-02 00:00:00,8.48295,3.69000,1,2,1,Day,6,0,0,0,0,0,0,0
57,2013-02-23 00:00:00,-0.16010,-3.32305,2,5,0,Day,58,0,0,0,0,0,0,0
107,2013-04-14 00:00:00,23.10085,10.59530,4,6,0,Day,108,0,0,1,0,0,0,0
518,2014-05-30 00:00:00,19.75755,11.63430,5,4,1,Day,519,0,0,0,0,0,0,0
601,2014-08-21 00:00:00,20.73565,9.97090,8,3,1,Day,602,0,0,0,0,0,0,0


In [86]:
df_labels.loc[:,'frequency'] = outliers_df.frequency

In [87]:
df_labels.to_csv(root_path +'detected_outliers.csv')

In [107]:
# finding top 5 outliers 
c = 10/len(embedding)
clf = KNN(n_neighbors=1, contamination=c)
top_outliers = pd.read_csv(root_path + 'df_labels.tsv', sep='\t').iloc[np.where(clf.fit_predict(embedding_norm) != 0)[0]] 
print('With the {} we obtain {} outliers'.format('KNN classifier',len(top_outliers)))
top_outliers

With the KNN classifier we obtain 10 outliers


,Date,MaxTemperature,MinTemperature,Month,WeekDay,is_WeekDay,Holiday,Index,OddWeekday,OddHoliday,OddTemp,OddNeighbor,HD_predicted,nonWorkingDay,ToTag
107,2013-04-14 00:00:00,23.10085,10.595300,4,6,0,Day,108,0,0,1,0,0,0,0
518,2014-05-30 00:00:00,19.75755,11.634300,5,4,1,Day,519,0,0,0,0,0,0,0
601,2014-08-21 00:00:00,20.73565,9.970900,8,3,1,Day,602,0,0,0,0,0,0,0
730,2014-12-28 00:00:00,2.54220,-2.465525,12,6,0,Day,731,1,0,1,1,0,0,0
929,2015-07-15 00:00:00,29.93275,17.079400,7,2,1,Day,930,0,0,0,0,0,0,0
950,2015-08-05 00:00:00,29.80220,15.298600,8,2,1,Day,951,0,0,0,0,0,0,0
956,2015-08-11 00:00:00,27.72720,17.603650,8,1,1,Day,957,0,0,0,1,0,0,0
957,2015-08-12 00:00:00,31.32290,19.231900,8,2,1,Day,958,0,0,0,0,0,0,0
1225,2016-05-06 00:00:00,22.15870,9.751750,5,4,1,Day,1226,0,0,0,1,0,0,0
1482,2017-01-18 00:00:00,1.42690,-3.883100,1,2,1,Day,1483,0,0,0,0,0,0,0


In [100]:
print(top_outliers[top_outliers.is_WeekDay == 0 ])

                    Date  MaxTemperature  MinTemperature  Month  WeekDay  \
57   2013-02-23 00:00:00        -0.16010       -3.323050      2        5   
107  2013-04-14 00:00:00        23.10085       10.595300      4        6   
730  2014-12-28 00:00:00         2.54220       -2.465525     12        6   

     is_WeekDay Holiday  Index  OddWeekday  OddHoliday  OddTemp  OddNeighbor  \
57            0     Day     58           0           0        0            0   
107           0     Day    108           0           0        1            0   
730           0     Day    731           1           0        1            1   

     HD_predicted  nonWorkingDay  ToTag  
57              0              0      0  
107             0              0      0  
730             0              0      0  


In [104]:
metadata_df = pd.read_csv(root_path + 'df_labels.tsv', sep='\t').drop(columns=['Index'])
metadata_df.head()


,Date,MaxTemperature,MinTemperature,Month,WeekDay,is_WeekDay,Holiday,OddWeekday,OddHoliday,OddTemp,OddNeighbor,HD_predicted,nonWorkingDay,ToTag
0,2012-12-28 00:00:00,11.31075,7.796150,12,4,1,Day,0,0,0,0,0,0,0
1,2012-12-29 00:00:00,12.78590,6.266150,12,5,0,Day,1,0,0,0,0,0,0
2,2012-12-30 00:00:00,10.73655,6.287500,12,6,0,Day,0,0,0,0,0,0,0
3,2012-12-31 00:00:00,11.01060,6.020600,12,0,1,Holiday,0,1,0,0,1,0,0
4,2013-01-01 00:00:00,8.94890,4.689225,1,1,1,Holiday,0,1,0,0,1,0,0


In [116]:
c = 5/len(embedding_norm)
clf = KNN(n_neighbors=1, contamination=c)
outliers = metadata_df.iloc[np.where(clf.fit_predict(embedding_norm) != 0)[0]] 
outliers

,Date,MaxTemperature,MinTemperature,Month,WeekDay,is_WeekDay,Holiday,OddWeekday,OddHoliday,OddTemp,OddNeighbor,HD_predicted,nonWorkingDay,ToTag
518,2014-05-30 00:00:00,19.75755,11.63430,5,4,1,Day,0,0,0,0,0,0,0
601,2014-08-21 00:00:00,20.73565,9.97090,8,3,1,Day,0,0,0,0,0,0,0
929,2015-07-15 00:00:00,29.93275,17.07940,7,2,1,Day,0,0,0,0,0,0,0
957,2015-08-12 00:00:00,31.32290,19.23190,8,2,1,Day,0,0,0,0,0,0,0
1225,2016-05-06 00:00:00,22.15870,9.75175,5,4,1,Day,0,0,0,1,0,0,0
